In [1]:
"""
Extract attributes from Google Earth Engine (GEE)

maxwell.cook@colorado.edu
"""

import ee
import geemap
import time

ee.Authenticate()

ee.Initialize(project='jfsp-aspen')

print("Success")

Success


In [2]:
def monitor_gee_export(task, sleep=30):
    """ Monitors EE export task """
    while task.active():
        print('Waiting for export to finish..\n\tPatience young padawan.')
        time.sleep(sleep)  # Check every 30 seconds
    
    # Get the status of the task
    status = task.status()
    
    # Check if the task failed or succeeded
    if status['state'] == 'COMPLETED':
        print("Export completed successfully !!!!")
    elif status['state'] == 'FAILED':
        print(f"Export failed! Bummer. Reason: {status.get('error_message', 'Unknown error')}")
    else:
        print(f"Export ended with state: {status['state']}")

print("Functions ready!")

Functions ready!


# Calculate Sentinel-based MNDWI, EVI, and LAI for fire bounds

Bring in the fire bounds (representing bounding geometry for AFD data within each fire). Calculate the pre-fire Sentinel-2 metrics.

In [3]:
# Load the fire bounds
bounds = ee.FeatureCollection('projects/jfsp-aspen/assets/afd_aspen-fires_2018_to_2023_bounds')
print(bounds.first().propertyNames().getInfo())
# Grab a list of fire IDs
fire_ids = bounds.aggregate_array('Fire_ID').distinct()
print(fire_ids.slice(0,10).getInfo())

['Fire_Year', 'Fire_ID', 'Start_Day', 'Fire_Name', 'WF_CESSATI', 'End_Day', 'DISCOVERY_', 'system:index']
['13378', '13494', '14', '15982', '16001', '16002', '16006', '16010', '16076', '22738']


In [4]:
# Clean up some names
bounds = bounds.select(
    ['Fire_ID', 'Fire_Name', 'Fire_Year', 'DISCOVERY_', 'WF_CESSATI'],
    ['Fire_ID', 'Fire_Name', 'Fire_Year', 'Ign_DOY', 'Last_DOY']
)
print(bounds.first().propertyNames().getInfo())    

['system:index', 'Fire_Year', 'Fire_ID', 'Last_DOY', 'Fire_Name', 'Ign_DOY']


In [5]:
# Grab a list of Sentinel-2 bands we will need for indices
# Keep RGB for visualization of the composites
s2_bands = ['B2', 'B3', 'B4', 'B8', 'B11']
print(f"Extracting S2 bands: {s2_bands}")

Extracting S2 bands: ['B2', 'B3', 'B4', 'B8', 'B11']


In [6]:
def generate_composites(geom_id):
    """ 
    Function to create a cloud-free image composite within polygon bounds
    Operates on a unique identifier (Fire ID)
    Args:
    - geom_id: the unique ID for polygon data within which to create the composite image
    """
    
    fire = bounds.filter(ee.Filter.eq('Fire_ID', geom_id)).first() # get the fire bounds
    fire_id = fire.get('Fire_ID') # get the Fire ID
    
    ig_year = ee.Number.parse(fire.get('Fire_Year')) # fire year
    start_doy = ee.Number.parse(fire.get('Ign_DOY')).subtract(60) # 60 days pre-ignition
    end_doy = ee.Number.parse(fire.get('Ign_DOY')).subtract(1) # day brfore ignition
    
    # Filter the S2-MSI collection
    s2_filter = ee.Filter.And(
      ee.Filter.bounds(bounds), # filter to the fire bounds
      ee.Filter.calendarRange(ee.Number(ig_year), ee.Number(ig_year), 'year'), # fire year
      ee.Filter.calendarRange(start_doy, end_doy, 'DAY_OF_YEAR'), # 60 days pre-ignition
      ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 60) # remove the most cloudy days
    )
    
    # Image Collection
    s2 = s2l1c.filter(s2_filter).select(s2_bands) # only keep the needed bands for indices
    
    # Grab some statistics from the collection
    n = s2.size();
    start_date = ee.Date(s2.first().get('system:time_start'));
    end_date = ee.Date(s2.limit(1,'system:time_start',False).first().get('system:time_start'));
    
    # Join to the Cloud Score Plus
    csPlus_s = csPlus.filter(
    ee.Filter.And(
      ee.Filter.bounds(bounds), # filter to the fire bounds
      ee.Filter.calendarRange(ig_year, ig_year, 'year'), # fire year
      ee.Filter.calendarRange(start_doy, end_doy, 'DAY_OF_YEAR'), # 60 days pre-ignition
    )
    ).select([QA_BAND]) # select the quality band
    
    # Mask out occluded pixels
    s2 = join_collections(s2, csPlus_s)
    s2 = s2.map(
      lambda img: img.updateMask(img.select(QA_BAND).gte(CLEAR_THRESHOLD)) # > CLEAR_THRESHOLD == occluded
    ).select(s2_bands)
    
    # Calculate the spectral indices, create the median composite, clip to bounds
    s2 = s2.map(calc_indices).median().clip(bounds)
    # Select the indices and visible bands
    s2 = s2.select(['B2','B3','B4','MNDWI','EVI','LAI'])
    
    return s2


def calc_indices(image):
    """ Calculates spectral indices from Sentinel-2 image data """
    
    # Modified Normalized Difference Water Index (MNDWI)
    # https://www.mdpi.com/2072-4292/8/4/354
    mndwi = image.normalizedDifference(['B3', 'B11']).rename('MNDWI')

    # Enhanced Vegetation Index (EVI)
    evi = image.expression(
        "2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))", {
            'NIR': image.select('B8'),
            'RED': image.select('B4'),
            'BLUE': image.select('B3')
        }).rename('EVI')

    image = image.addBands([mndwi, evi])

    # Leaf Area Index (LAI)
    lai = image.expression(
        '(3.618 * EVI - 0.118)', {
            'EVI': image.select('EVI')
        }).rename('LAI')

    return image.addBands(lai)


def string_cat(item, y):
    """ Function to concatenate strings (rename bands) """
    return ee.String(item).cat(y)

def apply_string_cat(x, y):
    return x.map(lambda item: string_cat(item, y))

# Function to join collections
def join_collections(col1, col2):
    joined = ee.ImageCollection(ee.Join.saveFirst('cs').apply(
        primary=col1,
        secondary=col2,
        condition=ee.Filter.equals(
            leftField='system:index',
            rightField='system:index'
        )
    ))
    return joined.map(lambda image: image.addBands(image.get('cs')))

print("Functions ready !")

Functions ready !


In [ ]:
# Extract the Sentinel-2 imagery for each fire.

In [10]:
# Load the S2-MSI Level 1C and cloud score plus
s2l1c = ee.ImageCollection("COPERNICUS/S2_HARMONIZED")
csPlus = ee.ImageCollection("GOOGLE/CLOUD_SCORE_PLUS/V1/S2_HARMONIZED")

QA_BAND = 'cs' # cloud score band
CLEAR_THRESHOLD = 0.60 # occluded pixel threshold

# Get the fire IDs
fire_ids = bounds.aggregate_array('Fire_ID').distinct()

# Run the function for all fires
composites = fire_ids.map(lambda x: generate_composites(x))

# Create the image collection
img_col = ee.ImageCollection.fromImages(composites)
# print(f"Band names: {img_col.first().bandNames().getInfo()}") # timeout on full dataset

# Create a mosaic image for export.
mosaic = img_col.mosaic()
# print(mosaic.bandNames().getInfo()) # timeout on full dataset

print("Done !")

Done !


In [ ]:
# Initialize a map (timeout on full dataset)
Map = geemap.Map()
Map.addLayerControl()

# Vis params
vis_params_lai = {
    'bands': ['LAI'],
    'min': -50,
    'max': 100,
}
vis_params_mndwi = {
    'bands': ['MNDWI'],
    'min': -1,
    'max': 1,
}
vis_params_rgb = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 3000,
}

# Add the layers to the map
Map.addLayer(mosaic, vis_params_lai, "LAI")
Map.addLayer(mosaic, vis_params_mndwi, "MNDWI")
Map.addLayer(mosaic, vis_params_rgb, "RGB")

Map

In [ ]:
# Export the mosaic imagery.

bands_ = ['MNDWI','LAI','EVI']
mosaic_ = mosaic.select(bands_)

task = ee.batch.Export.image.toAsset(
    image=mosaic_,  
    description='afd_aspen-fires_S2metrics_mosaic', 
    assetId='projects/jfsp-aspen/assets/afd_aspen_fires_S2metrics_mosaic', 
    region=bounds.geometry().bounds(), 
    scale=30, 
    crs="EPSG:4326", 
    maxPixels=1e13
)

# Start the export task
task.start()
print("Export to Earth Engine Asset started!")
# Monitor the task until it's finished
monitor_gee_export(task, 300) # print every 5 min

Export to Earth Engine Asset started!
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting fo

In [ ]:
# Calculate statistics in AFD observations

In [ ]:
# Load the exported asset from above
s2metrics = ee.Image('projects/jfsp-aspen/assets/afd_aspen_fires_S2metrics_mosaic')

In [ ]:
afds = ee.FeatureCollection('projects/jfsp-aspen/assets/AFD/combined-afd_aspen-fires_2018_to_2023')
print(afds.first().propertyNames().getInfo())

In [ ]:
# Simplify the feature collection properties.
afds = afds.select(['afdID'])

# Perform the reductions.

mean_med_stdDev = ee.Reducer.mean().combine(
    reducer2=ee.Reducer.stdDev(), sharedInputs=True).combine(
    reducer2=ee.Reducer.median(), sharedInputs=True)

# Perform the reduction by AFD observations
s2_stats = mosaic_.reduceRegions(
    collection=afds,
    reducer=mean_med_stdDev, # Mean and Standard Deviation CBI, CBI_bc, rbr
    scale=30
)

print("Submitted !")

In [ ]:
# Export the table. 
s2_stats = s2_stats.map(lambda feature: feature.setGeometry(None)) # drop geometry column

export_task = ee.batch.Export.table.toDrive(
    collection=s2_stats,
    description='afd_aspen-fires_s2-stats',
    fileFormat='CSV', 
    fileNamePrefix='afd_aspen-fires_s2-stats',
    folder='S2'
)

# Start the export task
export_task.start()
print("Export to Earth Engine Asset started!")
# Monitor the task until it's finished
monitor_gee_export(export_task, 120) # print every 2 min

In [ ]:
gc.collect()